<img src="images/logo.png" width="300">

In [1]:
#import modules

import numpy as np
import pandas as pd
import ipywidgets as widg
import IPython.display as idisp
import plotly.graph_objects as go


In [2]:
#color functions

predify_color = '0,71,187' #Predify dark blue

#print markdown text in color
def html( prefix, text ):
    idisp.display( idisp.Markdown( '{} <span style="color: rgb({})">{}</span>'.format( prefix, predify_color, text ) ) )


In [3]:
html( '#', 'Análisis de spots (Auditsa) y ratings (Inra) en la radio mexicana' )

# <span style="color: rgb(0,71,187)">Análisis de spots (Auditsa) y ratings (Inra) en la radio mexicana</span>

In [4]:
#initial settings

loadflag = 'y' #load files ('y') or calculate from scratch ('n')

dataloc = 'data/' #location of raw datafiles
imgloc = 'images/' #location of images

#plot settings

months = [ 'enero', 'febrero', 'marzo', 'abril', 'mayo' ] #months in right order

plot_props = { 'marker' : 'o',
    'ms' : 12,
    'ls' : '-',
    'lw' : 3,
    'fontsize' : 20,
    'xylabel' : 20,
    'ticklabel' : 15,
    'legend_prop' : { 'size':15 },
    'legend_hlen' : 2,
    'legend_np' : 2 }


In [5]:
#data loading

savename = dataloc+'data.h5' #HDF5 savename

data = {} #raw data
data['auditsa'] = pd.read_hdf( savename, key='auditsa' )
data['inra'] = pd.read_hdf( savename, key='inra' )


In [6]:
html( '###', 'Columnas en datos' )

### <span style="color: rgb(0,71,187)">Columnas en datos</span>

In [7]:
#get columns in dataset

def columns_dataset( base ):
    for col in data[ base ].columns:
        print(col)
        
idisp.display( widg.interactive( columns_dataset, base=data.keys() ) ) #get columns in dataset


interactive(children=(Dropdown(description='base', options=('auditsa', 'inra'), value='auditsa'), Output()), _…

In [8]:
html( '###', 'Elementos más comunes' )

### <span style="color: rgb(0,71,187)">Elementos más comunes</span>

In [9]:
#get top elements in column

def top_elements_column( df, columna, top ):
    print( df[ columna ].value_counts()[:top] )

html( '####', 'Auditsa' )
    
idisp.display( widg.interactive( top_elements_column, df=widg.fixed( data['auditsa'] ),
                                 columna=data['auditsa'].columns, top=(1, 30) ) ) #get top elements in column


#### <span style="color: rgb(0,71,187)">Auditsa</span>

interactive(children=(Dropdown(description='columna', options=('categoria', 'compania', 'competencia', 'estaci…

In [10]:
html( '####', 'Inra' )

idisp.display( widg.interactive( top_elements_column, df=widg.fixed( data['inra'] ),
                                 columna=data['inra'].columns, top=(1, 30) ) ) #get top elements in column


#### <span style="color: rgb(0,71,187)">Inra</span>

interactive(children=(Dropdown(description='columna', options=('plaza', 'mes', 'estudio', 'dias', 'target', 'u…

In [11]:
html( '###', 'Estaciones por plaza' )

### <span style="color: rgb(0,71,187)">Estaciones por plaza</span>

In [12]:
#get stations per place

def stations_place( df, plaza ):
    group_plaza_auditsa = df.groupby( 'plaza' ).get_group( plaza )
    
    for estacion in group_plaza_auditsa['estacion'].sort_values().unique():
        print( estacion )

html( '####', 'Auditsa' )
        
idisp.display( widg.interactive( stations_place, df=widg.fixed( data['auditsa'] ),
                                 plaza=data['auditsa']['plaza'].unique() ) ) #get stations per place

#### <span style="color: rgb(0,71,187)">Auditsa</span>

interactive(children=(Dropdown(description='plaza', options=('Monterrey', 'Puebla', 'Ciudad de México', 'Cuern…

In [13]:
html( '####', 'Inra' )

idisp.display( widg.interactive( stations_place, df=widg.fixed( data['inra'] ),
                                 plaza=data['inra']['plaza'].unique() ) ) #get stations per place

#### <span style="color: rgb(0,71,187)">Inra</span>

interactive(children=(Dropdown(description='plaza', options=('Ciudad de México', 'Monterrey', 'Guadalajara', '…

In [14]:
html( '###', 'Dashboard Auditsa' )

### <span style="color: rgb(0,71,187)">Dashboard Auditsa</span>

In [15]:
#plot total number of spots in stations

#define widgets

#station 1
est_1_widg = widg.Dropdown( options=data['auditsa']['estacion'].sort_values().unique(),
                            value='CDMX XHMVS-FM - (102.5 FM) MVS 102.5',
                            description='Estación 1:' )
#station 2
est_2_widg = widg.Dropdown( options=data['auditsa']['estacion'].sort_values().unique(),
                            value='CDMX XHEXA-FM - (104.9 FM) Exa',
                            description='Estación 2:' )
#filter
filtro_widg = widg.widgets.SelectMultiple( options=data['auditsa']['filtro'].sort_values().unique(),
                                           value=data['auditsa']['filtro'].sort_values().unique().tolist(),
                                           description='Filtro:' )
#sector
sector_widg = widg.Dropdown( options=data['auditsa']['sector'].sort_values().dropna().unique(),
                             value='COMPUTACION',
                             description='Sector:' )

#update sector widget with filter
def update_sec_widg( *args ):
    sector_widg.options = data['auditsa'][ data['auditsa']['filtro'].isin( filtro_widg.value ) ]['sector'].sort_values().unique()

#plot total number of spots in stations
def plot_spots_station( estacion_1, estacion_2, filtro, sector ):
    
    cond_1 = ( ( data['auditsa']['filtro'].isin( filtro ) )
             & ( data['auditsa']['sector'] == sector )
             & ( data['auditsa']['estacion'] == estacion_1 )
             ) #condition to filter data
    yplot_1 = data['auditsa'][ cond_1 ].groupby( 'mes' )['hits'].agg( np.sum ).reindex( months ) #aggregate by month

    cond_2 = ( ( data['auditsa']['filtro'].isin( filtro ) )
             & ( data['auditsa']['sector'] == sector )
             & ( data['auditsa']['estacion'] == estacion_2 )
             ) #condition to filter data
    yplot_2 = data['auditsa'][ cond_2 ].groupby( 'mes' )['hits'].agg( np.sum ).reindex( months ) #aggregate by month
    
    #initialise plot
    fig = go.Figure()
    fig.update_layout( title='Número de spots en estaciones',
                       xaxis_title='mes',
                       yaxis_title='Número de spots' )
    
    #plot plot!
    fig.add_trace( go.Scatter( x=yplot_1.index, y=yplot_1, name=estacion_1 ) )
    fig.add_trace( go.Scatter( x=yplot_2.index, y=yplot_2, name=estacion_2 ) )
    
    #finalise plot
    fig.show()

#observe widgets
filtro_widg.observe( update_sec_widg, 'value' ) #sector widget
    
#display interactive widget
idisp.display( widg.interactive( plot_spots_station, estacion_1=est_1_widg, estacion_2=est_2_widg,
                                 filtro=filtro_widg, sector=sector_widg ) )

interactive(children=(Dropdown(description='Estación 1:', index=49, options=('CDMX XEABC-AM - (760 AM) ABC Rad…

In [16]:
html( '###', 'Dashboard Inra' )

### <span style="color: rgb(0,71,187)">Dashboard Inra</span>

In [17]:
#plot audience in month for all stations

#define widgets

#station 1
mes_1_widg = widg.Dropdown( options=months,
                            value='marzo',
                            description='Mes 1:' )
#station 2
mes_2_widg = widg.Dropdown( options=months,
                            value='abril',
                            description='Mes 2:' )
#days
dias_widg = widg.Dropdown( options=data['inra']['dias'].sort_values().unique(),
                                value='L-V',
                                description='Días:' )
#place
plaza_widg = widg.SelectMultiple( options=data['inra']['plaza'].sort_values().unique(),
                            value=['Ciudad de México'],
                            description='Plaza:' )
#target
target_widg = widg.RadioButtons( options=data['inra']['target'].sort_values().unique(),
                                 value='Total Población',
                                 description='Target:' )

#plot audience in month for all stations
def plot_audience_month( mes_1, mes_2, dias, plaza, target ):

    cond_1 = ( ( data['inra']['mes'] == mes_1 )
             & ( data['inra']['dias'] == dias )
             & ( data['inra']['plaza'].isin( plaza ) )
             & ( data['inra']['target'] == target )
             )
    yplot_1 = data['inra'][ cond_1 ].groupby( 'estacion' )['audiencia'].agg( np.sum ).sort_values( ascending=False ) #aggregate by station

    cond_2 = ( ( data['inra']['mes'] == mes_2 )
             & ( data['inra']['dias'] == dias )
             & ( data['inra']['plaza'].isin( plaza ) )
             & ( data['inra']['target'] == target )
             )
    yplot_2 = data['inra'][ cond_2 ].groupby( 'estacion' )['audiencia'].agg( np.sum ).sort_values( ascending=False ) #aggregate by station

    #initialise plot
    fig = go.Figure()
    fig.update_layout( title='Audiencia por estación',
                       xaxis_title='Estación',
                       yaxis_title='Audiencia' )

    #plot plot!
    fig.add_trace( go.Bar( x=yplot_1.index, y=yplot_1, name=mes_1 ) )
    fig.add_trace( go.Bar( x=yplot_2.index, y=yplot_2, name=mes_2 ) )
    
    #finalise plot
    fig.update_layout( barmode='group', xaxis_tickangle=-45 )
    fig.show()

#display interactive widget
idisp.display( widg.interactive( plot_audience_month, mes_1=mes_1_widg, mes_2=mes_2_widg, dias=dias_widg,
                                 plaza=plaza_widg, target=target_widg ) )

interactive(children=(Dropdown(description='Mes 1:', index=2, options=('enero', 'febrero', 'marzo', 'abril', '…

In [21]:
#plot size of audience per hour

#define widgets

#radio group 1
comp_1_widg = widg.Dropdown( options=data['inra']['competencia'].sort_values().unique(),
                            value='MVS Noticias',
                            description='Grupo 1:' )
#station 1
est_1_widg = widg.Dropdown( options=data['inra'][ data['inra']['competencia'] == 'MVS Noticias' ]['estacion'].sort_values().unique(),
                            value='MVS NOTICIAS',
                            description='Estación 1:' )
#radio group 2
comp_2_widg = widg.Dropdown( options=data['inra']['competencia'].sort_values().unique(),
                            value='Exa',
                            description='Grupo 2:' )
#station 2
est_2_widg = widg.Dropdown( options=data['inra'][ data['inra']['competencia'] == 'Exa' ]['estacion'].sort_values().unique(),
                            value='EXA',
                            description='Estación 2:' )
#days
dias_widg = widg.Dropdown( options=data['inra']['dias'].sort_values().unique(),
                                value='L-V',
                                description='Días:' )
#target
target_widg = widg.RadioButtons( options=data['inra']['target'].sort_values().unique(),
                                 value='Total Población',
                                 description='Target:' )

#update station 1 widget with radio group 1
def update_est1_widg( *args ):
    est_1_widg.options = data['inra'][ data['inra']['competencia'] == comp_1_widg.value ]['estacion'].sort_values().unique()

#update station 2 widget with radio group 2
def update_est2_widg( *args ):
    est_2_widg.options = data['inra'][ data['inra']['competencia'] == comp_2_widg.value ]['estacion'].sort_values().unique()

#plot size of audience per hour
def plot_audience_hour( comp_1, estacion_1, comp_2, estacion_2, dias, target ):
    
    cond_1 = ( ( data['inra']['estacion'] == estacion_1 )
             & ( data['inra']['dias'] == dias )
             & ( data['inra']['target'] == target )
             ) #condition to filter data
    yplot_1 = data['inra'][ cond_1 ].groupby( 'hora' )['audiencia'].agg( np.sum ).sort_index() #aggregate by hours

    cond_2 = ( ( data['inra']['estacion'] == estacion_2 )
             & ( data['inra']['dias'] == dias )
             & ( data['inra']['target'] == target )
             ) #condition to filter data
    yplot_2 = data['inra'][ cond_2 ].groupby( 'hora' )['audiencia'].agg( np.sum ).sort_index() #aggregate by hours
    
    #initialise plot
    fig = go.Figure()
    fig.update_layout( title='Audiencia por hora',
                       xaxis_title='Hora',
                       yaxis_title='Audiencia' )
    
    #plot plot!
    fig.add_trace( go.Scatter( x=yplot_1.index, y=yplot_1, name=estacion_1 ) )
    fig.add_trace( go.Scatter( x=yplot_2.index, y=yplot_2, name=estacion_2 ) )
    
    #finalise plot
    fig.update_layout( hovermode='x' )
    fig.show()
    
#observe widgets
comp_1_widg.observe( update_est1_widg, 'value' ) #station1 widget
comp_2_widg.observe( update_est2_widg, 'value' ) #station1 widget
    
#display interactive widget
idisp.display( widg.interactive( plot_audience_hour, comp_1=comp_1_widg, estacion_1=est_1_widg,
                                 comp_2=comp_2_widg, estacion_2=est_2_widg,
                                 dias=dias_widg, target=target_widg ) )

interactive(children=(Dropdown(description='Grupo 1:', index=4, options=('Exa', 'FM Globo', 'INDIRECTA', 'La M…